In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob

from chromalab.observer import Observer, transformToChromaticity
from chromalab.spectra import Illuminant
from chromalab.maxbasis import MaxBasis
from chromalab.cubemap import CubeMap

In [ ]:
def plot_points(points, rgbs):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=rgbs)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.show()

def plotPointsDebug(points, rgbs=None):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.set_zlim(-1, 1)
    if rgbs is None:
        ax.scatter(points[:, 0], points[:, 1], points[:, 2])
    else:
        ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=rgbs)

    plt.show()

def plotAllSpectra(refs, rgbs):
    fig, ax = plt.subplots(4, 4)
    for idx, ref in enumerate(refs):
        i = idx // 4
        j = idx % 4
        ax[i][j].plot(ref.data, c=rgbs[idx])
        ax[i][j].set_ylim(-0.1, 1.1)
        ax[i][j].set_title(f"Spectra #{idx}")
    plt.tight_layout()
    plt.show()

def getnames(root):
    points_filename = f"../../../optimal-object-colors/Data/Printer-Data/{root}_ink_gamut_point_cloud2.npy"
    percentages_filename = f"../../../optimal-object-colors/Data/Printer-Data/{root}_ink_gamut_percentages2.npy"
    reflectances_filename = f"../../../optimal-object-colors/Data/Printer-Data/{root}_ink_gamut_reflectances2.npy"
    return points_filename, percentages_filename, reflectances_filename


In [ ]:
wavelengths = np.arange(390, 700, 10)
illuminant = Illuminant.get("D65")

tetrachromat = Observer.tetrachromat(wavelengths=wavelengths, illuminant=illuminant, verbose=True)
# points, rgbs = tetrachromat.get_optimal_colors()
# chrom_pts = transformToChromaticity(points)
# plot_points(chrom_pts, rgbs)
#%% 
maxbasis = MaxBasis(tetrachromat, verbose=True)
tetra_maxbasis = maxbasis.get_max_basis_observer()
# points, rgbs = tetra_maxbasis.get_optimal_colors()
# chrom_pts = transformToChromaticity(points)
# plot_points(chrom_pts, rgbs)

In [ ]:
points_filename, percentages_filename, reflectances_filename = getnames("measured")
spectra_wavelengths = np.arange(400, 701, 10)
pts = np.load(points_filename)
refs = np.load(reflectances_filename)
Tmat = maxbasis.get_cone_to_maxbasis_transform()
maxbasis_pts = (Tmat @ pts.T).T
chrom_pts = transformToChromaticity(maxbasis_pts)

fig, axs = plt.subplots(1, 2)

axs[0].hist(np.sum(maxbasis_pts, axis=1))
axs[0].set_title('Sum of Points')
axs[1].hist(np.linalg.norm(chrom_pts, axis=1))
axs[1].set_title('Norm of Chromaticity Points')
plt.show()

In [ ]:
basepath = "../../Data/Printer-Data/inkset"
volumes = []
for i in range(4, 5):
    filename = glob.glob(f"{basepath}{i}/*point_cloud5.npy")[0]
    points = np.load(chrom_pts)
    cubemap = CubeMap(chrom_pts, maxbasis, refs=refs.squeeze(), ref_wavelengths=spectra_wavelengths, verbose=True)
    cubemap.display_cube_map(1.5, 0.3, 9, lum_thresh=0.5, sat_thresh=0.25)